<a href="https://colab.research.google.com/github/Matabudi/new/blob/main/BUNKR_DOWNLOAD%26DOWNSCALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BUNKR VIDEO DOWNLOADER & COMPRESSOR**
##### This notebook allows you to download videos from Bunkr using a provided URL and compress them to a lower resolution to reduce file size. The process includes connecting to Google Drive for storage, leveraging GPU for faster video compression, and providing a progress bar for visual feedback.

### **MOUNT GOOGLE DRIVE**

In [2]:
# @markdown ##### Mount Google Drive if you intend to save your downloads to your drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **INSTALL DEPENDENCIES**

In [ ]:
# @markdown #####Install the required libraries and  with GPU support.
!pip install tqdm
!pip install ffmpeg-python
!apt update -qq && apt install -y ffmpeg
!pip install --upgrade --force-reinstall --no-deps 'git+https://github.com/kkroening/ffmpeg-python.git'

# Ensure CUDA and NVIDIA drivers are installed
!apt-get install -qq cuda

from IPython.display import display, HTML, clear_output

46 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
  Cloning https://github.com/kkroening/ffmpeg-python.git to /tmp/pip-req-build-nyv8jjim
  Running command git clone --filter=blob:none --quiet https://github.com/kkroening/ffmpeg-python.git /tmp/pip-req-build-nyv8jjim
  Resolved https://github.com/kkroening/ffmpeg-python.git to commit df129c7ba30aaa9ffffb81a48f53aa7253b0b4e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ffmpeg-python: filename=ffmpeg_python-0.2.0-py3-none-any.whl size=25327 sha256=cc57ac6c8e95c39b37085de51dd776547e4a702b581c6b30278be8ce1df9fb36
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nrdxyk3/wheels/0c/93/fb/4f6

### **DOWNLOAD & COMPRESS VIDEO FILE**
##### Paste the video link, select the download destination, and choose the video resolution.

##### **Note that if you choose to compress the video, it will be renamed to downscaled_video.mp4.**

In [7]:

# Define the video URL
# @markdown ### **Video Link**
# @markdown ##### *Paste a valid URL link of the video you wish to download into the field below;*
video_url = 'https://kebab.bunkr.ru/IMG_0287-lnox2eiH-yiq5gazq.mov'  # @param {type:"string", label:"Video Link"}

# Define the save path
# @markdown ### **Save Location**
# @markdown ##### *You can select either 'gdrive' option to save the file in Google Drive or select the 'colab' option to save the file in the Colab directory.*
save_option = "gdrive"  # @param ["gdrive", "colab"] {type:"string"}

# Path based on the environment
if save_option == "gdrive":
    save_path = '/content/drive/MyDrive/video.mp4'  # Google Drive path
else:
    save_path = '/content/video.mp4'  # Colab default path

In [8]:
# @markdown ### **Download Video**
import requests
from tqdm import tqdm
import os

def download_video(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(save_path, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    if total_size != 0 and t.n != total_size:
        print("ERROR, something went wrong")
    else:
        print(f"Video downloaded to {save_path}")

# Execute download
download_video(video_url, save_path)

100%|██████████| 224M/224M [00:11<00:00, 18.9MiB/s]

Video downloaded to /content/drive/MyDrive/video.mp4


In [9]:
import ffmpeg
import os

# Define the desired resolution
# @markdown ### **Downscaled Resolution**
# @markdown ###### *Select the resolution for compression*
desired_resolution = "480p"  # @param ["480p", "360p"] {type:"string"}

def downscale_video(input_path, resolution="480p"):
    try:
        # Map resolution options to scale values
        resolution_map = {
            "480p": "854x480",
            "360p": "640x360"
        }

        if resolution not in resolution_map:
            raise ValueError("Invalid resolution. Choose either '480p' or '360p'.")

        scale_value = resolution_map[resolution]

        # Define the output video path
        output_path = input_path.replace('.mp4', f'_{resolution}.mp4')

        # Probe the video to get file info
        probe = ffmpeg.probe(input_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        total_frames = int(video_info['nb_frames'])

        # Function to update the progress bar
        def progress_callback(stats):
            frame = int(stats.get('frame', 0))
            t.update(frame - t.n)

        # Create a progress bar
        with tqdm(total=total_frames, unit='frame') as t:
            # Use h264_nvenc codec for GPU encoding
            process = (
                ffmpeg
                .input(input_path)
                .output(output_path, vf=f"scale={scale_value}", vcodec="h264_nvenc", preset="fast")
                .global_args('-progress', 'pipe:1', '-nostats')
                .global_args('-y')  # Overwrite output files without asking
                .global_args('-loglevel', 'error')  # Suppress ffmpeg console output
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
            )
            for line in process.stdout:
                line = line.decode().strip()
                if 'frame=' in line:
                    stats = {}
                    for item in line.split(' '):
                        key, value = item.split('=')
                        stats[key] = value
                    progress_callback(stats)

        print(f"Downscaled video saved to {output_path}")

        # Print original and downscaled file sizes
        original_size = os.path.getsize(input_path) / (1024 * 1024)
        downscaled_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"Original size: {original_size:.2f} MB")
        print(f"Downscaled size: {downscaled_size:.2f} MB")

    except ffmpeg.Error as e:
        # Print the full error message for debugging
        print(f"ffmpeg error: {e.stderr.decode()}")
    except ValueError as ve:
        # Handle invalid resolution error
        print(ve)

# Execute compression
downscale_video(save_path, resolution=desired_resolution)

4967frame [00:49, 100.36frame/s]

Downscaled video saved to /content/drive/MyDrive/video_downscaled.mp4
Original size: 213.61 MB
Downscaled size: 21.08 MB


## **STREAM VIDEO**
##### Stream the downloaded video file.

In [10]:

# Define the video file location for streaming
# @markdown ### **Video Location for Streaming**
# @markdown ##### *Select either 'gdrive' or 'colab' based on where the video is saved.*
stream_option = "gdrive"  # @param ["gdrive", "colab"] {type:"string"}

# Path based on the environment
if stream_option == "gdrive":
    video_file = '/content/drive/MyDrive/video.mp4'  # Google Drive path
else:
    video_file = '/content/video.mp4'  # Colab default path

# Stream the video
from IPython.display import display, HTML

display(HTML(f"""
<video width="640" height="480" controls>
  <source src="{video_file}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""))